In [ ]:
import torch
import pandas
import matplotlib.pyplot as plt
import os
import import_ipynb

os.getcwd()

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import getDataset

S = 7
B = 2
C = 20

# csv_directory, images_directory, labels_directory, transformation
csv_dir  = 'D:\\WORK\\DATASETS\\archive\\train.csv'
img_dir  = 'D:\\WORK\\DATASETS\\archive\\images'
lbl_dir  = 'D:\\WORK\\DATASETS\\archive\\labels'

to = [T.ToTensor(),T.Resize((448,448))]
dataset  = getDataset.load_ds(csv_dir,
                              img_dir,
                              lbl_dir,
                              transform=to, S=S, B=B, C=C)
print("Dataset length: ", len(dataset))

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size  = 5
num_workers = 0
train_dl    = DataLoader(dataset, batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

In [ ]:
import architecture

S = 7
B = 2
C = 20

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model  = architecture.model(S=S, B=B, C=C).to(device)
model.parameters()

# x = torch.randn((2, 3, 448, 448))    

In [ ]:
for x, y in train_dl:
    print(x.shape, y.shape)
#     print(model(x).reshape(-1, 7, 7, 30).shape)
    break

In [ ]:
from tqdm import tqdm
import torch.optim as optim
from loss import current_loss

loop      = tqdm(enumerate(train_dl), total=len(train_dl), leave=False)
myloss    = current_loss(S, B, C, 'midpoint')
avg_loss  = []
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0)

print("Using :", device)
for batch_idx, (x, y) in loop:
    x = x.to(device)
    y = y.to(device)
    
    loss = myloss.fwd(model(x), y)
    print(loss)
    avg_loss.append(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loop.set_postfix(loss=loss.item())

print(f"Mean loss was {sum(avg_loss)/len(avg_loss)}")
    
    
    